# installs & imports

In [1]:
import os
from os.path import join
import json
import openai
from IPython.display import display, Markdown

# chatOpenAI within langchain
from langchain.chat_models import ChatOpenAI

# prompt template refers to a reproducible way to generate a prompt
from langchain.prompts import ChatPromptTemplate

# LangChain for question answering over a list of documents
from langchain.chains import RetrievalQA

# Load data from a source as Document's. A Document is a piece of text and associated metadata.
from langchain.document_loaders import CSVLoader

# DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory
from langchain.vectorstores import DocArrayInMemorySearch


#takes care of storing embedded data and performing vector search for you
from langchain.indexes import VectorstoreIndexCreator

#pdf loader
from langchain.document_loaders import PyPDFLoader

# QA evaluator
from langchain.evaluation.qa import QAGenerateChain
# QA evaluates two responses
from langchain.evaluation.qa import QAEvalChain
# Load embeddings
from openai.api_resources.embedding import Embedding

# Load OpenAI Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

#Load LLMs
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

# functions & environment variables

In [2]:
CURRENT_PATH = os.getcwd()
CONFIG_FILE = "config.json"

# Load Config File
with open(join(CURRENT_PATH, CONFIG_FILE)) as file:
    # Load the JSON data
    config = json.load(file)

In [3]:
# Environmental Variables
AZURE_OPENAI_KEY = config['AZURE_OPENAI_KEY']
AZURE_OPENAI_ENDPOINT = config['AZURE_OPENAI_ENDPOINT']
AZURE_ENGINE_NAME = config['AZURE_ENGINE_NAME']
AZURE_ADA_NAME = config['AZURE_ADA_NAME']
AZURE_ADA_ENDPOINT = config['AZURE_ADA_ENDPOINT']
AZURE_ADA_KEY = config['AZURE_ADA_KEY']

In [4]:
# OpenAI API settings
openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

%env OPENAI_API_KEY=openai.api_key

env: OPENAI_API_KEY=openai.api_key


In [5]:
# OpenAI ADA API embeddings 
openai_ada_key = AZURE_ADA_KEY
openai_ada_base = AZURE_ADA_ENDPOINT
openai_ada_deployment = AZURE_ADA_NAME
openai_api_type = 'azure'


In [118]:
# gets an answer from an helpful assistant
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=AZURE_ENGINE_NAME,
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response

# load documents

In [6]:
HANDBOOK_PATH = 'onboarding docs'
HANDBOOK_FILE = 'handbook-6-10.pdf'

In [7]:
# load PDF File
HANDBOOK_PATH = 'onboarding docs'
HANDBOOK_FILE = 'handbook-6-10.pdf'
loader = PyPDFLoader(join(HANDBOOK_PATH,HANDBOOK_FILE))
data_handbook = loader.load()

In [12]:
embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                              , openai_api_base = openai_ada_base
                              , deployment = openai_ada_deployment
                              , openai_api_type = openai_api_type
                              , chunk_size = 1)

index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
                                , embedding= embedding ).from_loaders([loader])

llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
               )


example_gen_chain = QAGenerateChain.from_llm(llm)

questions = example_gen_chain.apply_and_parse([{"doc": t} for t in data_handbook[:5]])

In [121]:
qa = RetrievalQA.from_chain_type(llm,retriever=index.vectorstore.as_retriever(), chain_type="stuff"
                                       , verbose=True
                                       , chain_type_kwargs = {"document_separator": "<<<<>>>>>"} )

In [18]:
questions

[{'query': 'What are the five stages of the Universo card journey, and what is the sixth stage?',
  'answer': 'The five stages of the Universo card journey are from acquisition to collection, and the sixth stage is called delinquency management.'},
 {'query': 'What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?',
  'answer': 'Universo considers scoring applicational, rules of risk, rules of AML, and rules of documentation when deciding whether to assign a client a plafond and what is the maximum plafond a client can have is directly dependent on their scores and income levels. The plafond can be determined through a matrix of 15x7 levels of scores and income intervals. Clients with lower scores and income levels have more risky profiles and lower plafonds.'},
 {'query': 'What are the two systems core of Universo that are used to create a customer account with debit and credit payment options?',
  'ans

In [19]:
predictions = qa.apply(questions)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [21]:
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(questions, predictions)

In [22]:
graded_outputs = eval_chain.evaluate(questions, predictions)

In [133]:
def textwrap(s):
    new_s = ""
    words = s.split(" ")
    for i in range(len(words)):
        new_s += " " + words[i]
        if i % 8 == 0 and i >0:
            new_s += '\n'
    return new_s.strip().replace("\n ", "\n")

In [135]:
for question in predictions:
    question['answer'] = textwrap(question['answer']) 
    question['query'] = textwrap(question['query']) 
    question['result'] = textwrap(question['result']) 
    question['evaluation'] = graded_outputs[predictions.index(question)]['results']
  


In [132]:
graded_outputs

[{'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'}]

In [23]:
for i, eg in enumerate(questions):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: What are the five stages of the Universo card journey, and what is the sixth stage?
Real Answer: The five stages of the Universo card journey are from acquisition to collection, and the sixth stage is called delinquency management.
Predicted Answer: The five stages of the Universo card journey are: 
1. Angariação (Acquisition)
2. Produção/Envio do Cartão (Card Production/Shipment)
3. Ativação (Activation)
4. Utilização (Use)
5. Cobrança (Billing)

The sixth stage is called Gestão de Incumprimento (Debt Management), which occurs if the customer fails to make the monthly payment.
Predicted Grade: INCORRECT

Example 1:
Question: What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?
Real Answer: Universo considers scoring applicational, rules of risk, rules of AML, and rules of documentation when deciding whether to assign a client a plafond and what is the maximum plafond a client can 

In [138]:
a = [1,2,3,4,5]

In [24]:
answers = []
for question in questions:
    print(question['query'])
    answers.append(input())

What are the five stages of the Universo card journey, and what is the sixth stage?


 Acquisition, Onboarding, Transaction, Billing, Collection, Litigation


What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?


  It considers the credit score, if the customer has any problem with the Banco of Portugal, the income, the current expenses, and so on. 


What are the two systems core of Universo that are used to create a customer account with debit and credit payment options?


  Mastercard and Mambu


Who is responsible for the production and shipment of the cards mentioned in the document?


 MPTS for the production and CTT for the Shipment 


What are the monthly maintenance costs associated with using the Universo card?


  3DS, OTPs, billing process, Usage Rights


In [25]:
my_answers = []
for i in range(len(questions)):
    questions[i]['result'] = answers[i]
    my_answers.append(questions[i])

In [27]:
predictions

[{'query': 'What are the five stages of the Universo card journey, and what is the sixth stage?',
  'answer': 'The five stages of the Universo card journey are from acquisition to collection, and the sixth stage is called delinquency management.',
  'result': 'The five stages of the Universo card journey are: \n1. Angariação (Acquisition)\n2. Produção/Envio do Cartão (Card Production/Shipment)\n3. Ativação (Activation)\n4. Utilização (Use)\n5. Cobrança (Billing)\n\nThe sixth stage is called Gestão de Incumprimento (Debt Management), which occurs if the customer fails to make the monthly payment.'},
 {'query': 'What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?',
  'answer': 'Universo considers scoring applicational, rules of risk, rules of AML, and rules of documentation when deciding whether to assign a client a plafond and what is the maximum plafond a client can have is directly dependent on their

In [26]:
my_answers

[{'query': 'What are the five stages of the Universo card journey, and what is the sixth stage?',
  'answer': 'The five stages of the Universo card journey are from acquisition to collection, and the sixth stage is called delinquency management.',
  'result': 'Acquisition, Onboarding, Transaction, Billing, Collection, Litigation'},
 {'query': 'What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?',
  'answer': 'Universo considers scoring applicational, rules of risk, rules of AML, and rules of documentation when deciding whether to assign a client a plafond and what is the maximum plafond a client can have is directly dependent on their scores and income levels. The plafond can be determined through a matrix of 15x7 levels of scores and income intervals. Clients with lower scores and income levels have more risky profiles and lower plafonds.',
  'result': ' It considers the credit score, if the customer

In [28]:
graded_outputs_2 = eval_chain.evaluate(questions, my_answers)

In [29]:
for i, eg in enumerate(questions):
    print(f"Example {i}:")
    print("Question: " + my_answers[i]['query'])
    print("Real Answer: " + my_answers[i]['answer'])
    print("Predicted Answer: " + my_answers[i]['result'])
    print("Predicted Grade: " + graded_outputs_2[i]['results'])
    print()

Example 0:
Question: What are the five stages of the Universo card journey, and what is the sixth stage?
Real Answer: The five stages of the Universo card journey are from acquisition to collection, and the sixth stage is called delinquency management.
Predicted Answer: Acquisition, Onboarding, Transaction, Billing, Collection, Litigation
Predicted Grade: INCORRECT

Example 1:
Question: What factors does Universo consider when deciding whether to assign a client a plafond and what is the maximum plafond a client can have?
Real Answer: Universo considers scoring applicational, rules of risk, rules of AML, and rules of documentation when deciding whether to assign a client a plafond and what is the maximum plafond a client can have is directly dependent on their scores and income levels. The plafond can be determined through a matrix of 15x7 levels of scores and income intervals. Clients with lower scores and income levels have more risky profiles and lower plafonds.
Predicted Answer:  I

# Obtain for Valve

In [101]:
loader = PyPDFLoader("C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\Rizoma-DSSG\\.git\\aurora\\onboarding docs\\Valve_Handbook_LowRes.pdf")
data_handbook = loader.load()
data = loader.load()


embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                              , openai_api_base = openai_ada_base
                              , deployment = openai_ada_deployment
                              , openai_api_type = openai_api_type
                              , chunk_size = 1)

index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
                                , embedding= embedding ).from_loaders([loader])

llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                 ,openai_api_key=AZURE_OPENAI_KEY
                 ,openai_api_type='azure'
                 ,openai_api_version="2023-05-15"
                 ,deployment_name =AZURE_ENGINE_NAME
                 ,model = "gpt-3.5-turbo"
               )


example_gen_chain = QAGenerateChain.from_llm(llm)

questions = example_gen_chain.apply_and_parse([{"doc": t} for t in data_handbook])
for question in questions:
    question['answer'] = textwrap(question['answer']) 
    question['query'] = textwrap(question['query']) 
question_db.extend([pn.Row(questions, width=600 )])

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Cr

In [103]:

embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                      , openai_api_base = openai_ada_base
                      , deployment = openai_ada_deployment
                      , openai_api_type = openai_api_type
                      , chunk_size = 1)

# llm initiation
llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
         ,openai_api_key=AZURE_OPENAI_KEY
         ,openai_api_type='azure'
         ,openai_api_version="2023-05-15"
         ,deployment_name =AZURE_ENGINE_NAME
         ,model = "gpt-3.5-turbo"
       )

index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
                        , embedding= embedding ).from_loaders([loader])

qa = RetrievalQA.from_chain_type(llm,retriever=index.vectorstore.as_retriever(), chain_type="stuff"
                               , verbose=True
                               , chain_type_kwargs = {"document_separator": "<<<<>>>>>"} )
predictions = qa.apply(questions)


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 7 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Cr



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA ch

[{'query': 'What is the title of the document?',
  'answer': 'The title of the document is "HANDBOOK FOR NEW\nEMPLOYEES".',
  'result': 'The document is titled "Handbook for New Employees."'},
 {'query': 'What is the title of the document?',
  'answer': 'The title of the document is "HANDBOOK FOR NEW\nEMPLOYEES".',
  'result': 'The title of the document is "Handbook for New Employees."'},
 {'query': 'What is the purpose of the document?',
  'answer': 'The purpose of the document is to provide a\nhandbook for Valve employees and express gratitude to\ntheir families.',
  'result': 'The purpose of the document is to serve as a handbook for new employees at Valve, providing them with guiding principles and goals for the company. It also addresses potential challenges and encourages employees to be agents of change in order to align with these goals. Additionally, the document acknowledges areas where the company may have weaknesses and expresses a desire to improve in those areas.'},
 {'qu

In [121]:
t = []
res = {}

In [123]:
for p in predictions:
    res = {}
    res['query'] = p['query']
    res['answer'] = p['result']
    t.append(res)
    

In [124]:
t

[{'query': 'What is the title of the document?',
  'answer': 'The document is titled "Handbook for New Employees."'},
 {'query': 'What is the title of the document?',
  'answer': 'The title of the document is "Handbook for New Employees."'},
 {'query': 'What is the purpose of the document?',
  'answer': 'The purpose of the document is to serve as a handbook for new employees at Valve, providing them with guiding principles and goals for the company. It also addresses potential challenges and encourages employees to be agents of change in order to align with these goals. Additionally, the document acknowledges areas where the company may have weaknesses and expresses a desire to improve in those areas.'},
 {'query': 'According to the document, can the employment relationship between\nValve and an employee be terminated without cause\nor notice?',
  'answer': 'Yes, according to the document, the employment relationship between Valve and an employee can be terminated without cause or noti

In [ ]:
for question in predictions:
    question['answer'] = textwrap(question['result']) 
    question['query'] = textwrap(question['query']) 
    question['result'] = textwrap(question['result']) 
    question['evaluation'] = graded_outputs[predictions.index(question)]['results']

---------
# FRONT

In [41]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile

In [44]:
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)


openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Enter your OpenAI API Key here...", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type', 
    options=['stuff', 'map_reduce', "refine", "map_rerank"],
    value='map_reduce'
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),z
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings"
    ), width=670
)
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result
# os.environ["OPENAI_API_KEY"] = ""
# result = qa("materials/example.pdf", "When was GPT-2 created?", "map_reduce", 2)
convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value
    
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("temp.pdf")
    
        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n------------------------------------------------------- \
                        -------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
    return pn.Column(*convos, margin=15, width=575, min_height=400)
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=670, scroll=True)
# layout

In [46]:
CHAT_NAME = 'Aurora'

In [115]:
question_db = []

def qa_output(_):
    # load PDF File
    if file_input.value is not None:
        file_input.save("temp.pdf")

        loader = PyPDFLoader(file_input.value)
        data_handbook = loader.load()
    
        embedding = OpenAIEmbeddings(openai_api_key=openai_ada_key
                                      , openai_api_base = openai_ada_base
                                      , deployment = openai_ada_deployment
                                      , openai_api_type = openai_api_type
                                      , chunk_size = 1)
        
        index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch
                                        , embedding= embedding ).from_loaders([loader])
        
        llm = AzureChatOpenAI(openai_api_base=AZURE_OPENAI_ENDPOINT
                         ,openai_api_key=AZURE_OPENAI_KEY
                         ,openai_api_type='azure'
                         ,openai_api_version="2023-05-15"
                         ,deployment_name =AZURE_ENGINE_NAME
                         ,model = "gpt-3.5-turbo"
                       )
        
        
        example_gen_chain = QAGenerateChain.from_llm(llm)
        
        questions = example_gen_chain.apply_and_parse([{"doc": t} for t in data_handbook[:5]])
        question_db.extend([pn.Row(
                        pn.panel("\U0001F60A", width=10),
                        questions, width=600 )])

In [116]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')

introduction_eval = pn.Row(pn.pane.Markdown(f"""
#### My name is {CHAT_NAME} and I’m glad to generate some questions from your company’s existing documentation"""))

#file input
file_input = pn.widgets.FileInput(width=300)

# Button generates Q&A
run_qa = pn.widgets.Button(name="Generate Q&A!", styles=dict(background='White', text = 'white'), width=300)

# File Input + Button
generate_qa = pn.Row(file_input, run_qa)

# Q&A Page
generate_qa_interactive = pn.panel(pn.bind(qa_output, run_qa), loading_indicator=True)

# Q&A Output
output_qa = pn.WidgetBox('*Questions & Answers*', generate_qa_interactive, width=1100, scroll=True)

pn.Column(
    pn.pane.Markdown("""# Welcome to Aurora 🌌 """),
    pn.Card(introduction_eval
            , generate_qa
            , output_qa
            , collapsed = True
            , title='Create Onboarding Evaluation', styles=dict(background='White')),
    pn.pane.Markdown("""-----""")

).servable()


Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Card(collapsed=True, sizing_mode='stretch_width', styles={'background': 'White'}, title='Create Onboarding E...)
        [0] Row(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
        [1] Row(sizing_mode='stretch_width')
            [0] FileInput(width=300)
            [1] Button(name='Generate Q&A!', sizing_mode='stretch_width', styles={'background': 'White', ...})
        [2] WidgetBox(scroll=True, width=1100)
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] ParamFunction(function, _pane=Str, defer_load=False, loading_indicator=True, sizing_mode='stretch_width')
    [2] Markdown(str, sizing_mode='stretch_width')

In [120]:
file_input.value

In [119]:
question_db

[]